In [1]:
import json
import pickle
import numpy as np
import scipy
from scipy.special import softmax

## Prepare json file for attention extraction

In [8]:
sentences_file = '../data/europarl/intersect.en.tok.dev'
json_file = '../BertAA-dev/endev_4bert.json'

In [9]:
in_data = []
with open(sentences_file, 'r') as infile:
    for line in infile:
        in_data.append({'text': line.strip()})
        
with open(json_file, 'w') as outfile:
    json.dump(in_data, outfile)

In [10]:
len(in_data)

1000

## Load attention matrices

In [15]:
attention_pickle = '../BertAA/BertAA-dev/endev_4bert_akq.pkl'
def load_pickle(fname):
    with open(fname, "rb") as f:
        return pickle.load(f)  # add, encoding="latin1") if using python3 and downloaded data

# BERT-base Attention Maps extracted from Wikipedia
# Data is a list of dicts of the followig form:
# {
#    "tokens": list of strings
#    "attns": [n_layers, n_heads, n_tokens, n_tokens] 
#             tensor of attention weights
# }
data = load_pickle(attention_pickle)


## Processing of attention matrices
* The row and column coresponding to `[CLS]` token is deleted 
* The row and column coresponding to `[SEP]` is kept, so the format corresponds to attention metrices obtained in the previous experiments.
* Tokenization is changed to `BPE`. `[CLS]` and `[SEP]` tokens are deleted.
* Tokens are saved in a text file. One line per sentence.

In [16]:
output_attention = '../BertAA/BertAA-dev/attentions.npz'
output_tokens = '../BertAA/BertAA-dev/source.txt'

In [17]:
def to_bpe(tokens):
    txt_line = (' ').join(tokens)
    txt_line = txt_line.replace(' ##', '@@ ')
    return txt_line.split(' ')

In [18]:
attentions = []
sentences = []
for exmpl in data:
    tokens = to_bpe(exmpl['tokens'])
    sentence = (' ').join(tokens[1:-1])
    sentences.append(sentence)
    attentions.append(exmpl['attns'][:,:,1:,1:])
    

In [19]:
with open(output_attention, 'wb') as attn_file:
    np.savez(attn_file, *attentions)
    
with open(output_tokens, 'w') as src_file:
    src_file.write('\n'.join(sentences))
        

In [20]:
sentences[214]

'the rap@@ port@@ eur wants us to involve our national courts and ministries of justice in this process .'